In [1]:
!pip install requests beautifulsoup4



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# 필요한 라이브러리 가져오기
import requests  # 웹 페이지 요청을 보낼 때 사용
from bs4 import BeautifulSoup  # HTML을 파싱(분석)할 때 사용

def get_latest_news_url():
    # 뉴욕타임스 모닝 브리핑 페이지 URL
    url = "https://www.nytimes.com/newsletters/morning-briefing"
    
    # 해당 URL로 HTTP GET 요청을 보냄
    response = requests.get(url)

    # 만약 요청이 실패하면(응답 코드가 200이 아니면) 오류 메시지를 출력하고 None 반환
    if response.status_code != 200:
        print("❌ 뉴스 페이지 크롤링 실패")  # 실패 메시지 출력
        return None

    # HTML을 BeautifulSoup을 이용해 파싱
    soup = BeautifulSoup(response.text, "html.parser")

    # "Read the Latest"라는 텍스트를 가진 <a> 태그(버튼)를 찾음
    button = soup.find("a", string="Read the Latest")

    # 버튼이 존재하는 경우
    if button:
        latest_news_url = button["href"]  # <a> 태그의 href 속성(링크) 가져오기

        # 만약 링크가 상대 경로로 되어 있으면 절대 경로로 변환
        if latest_news_url.startswith("/"):
            latest_news_url = "https://www.nytimes.com" + latest_news_url  

        return latest_news_url  # 최신 뉴스 링크 반환

    else:
        # "Read the Latest" 버튼을 찾지 못한 경우 오류 메시지 출력
        print("❌ 'Read the Latest' 버튼을 찾을 수 없음")
        return None

# 실행 테스트: 함수 호출 후 반환된 최신 뉴스 URL 출력
latest_news_url = get_latest_news_url()
print("Latest News URL:", latest_news_url)


Latest News URL: https://static.nytimes.com/email-content/NN_sample.html


In [7]:
url = "https://www.nytimes.com/newsletters/morning-briefing"
    
# 해당 URL로 HTTP GET 요청을 보냄
response = requests.get(url)

response.text

'<!DOCTYPE html>\n<html lang="en"  xmlns:og="http://opengraphprotocol.org/schema/">\n  <head>\n    \n    \n    <meta charset="utf-8" />\n    <title data-rh="true">The Morning Newsletter - The New York Times</title>\n    <meta data-rh="true" property="og:url" content="https://www.nytimes.com/newsletters/morning-briefing"/><meta data-rh="true" property="og:title" content="Sign up for The Morning - The New York Times"/><meta data-rh="true" property="og:description" content="Make sense of the day’s news and ideas. Times journalists guide you through what’s happening — and why it matters."/><meta data-rh="true" property="og:type" content="article"/><meta data-rh="true" property="og:image" content="https://static.nytimes.com/email-images/NYT-Newsletters-TheMorning-Icon-500px.jpg"/><meta data-rh="true" name="description" content="Make sense of the day’s news and ideas. Times journalists guide you through what’s happening — and why it matters."/>  \n    <meta data-rh="true" name="twitter:card"

In [8]:
soup = BeautifulSoup(response.text, "html.parser")
soup.find("a", string="Read the Latest")

<a class="css-l2hx7w" href="https://static.nytimes.com/email-content/NN_sample.html" rel="noopener noreferrer" tabindex="-1" target="_blank">Read the Latest</a>

In [5]:
REST_API_KEY = "a3f4ffdc5ff3698dfa560ece7db9ab0e"
REDIRECT_URI = "https://localhost"

AUTH_URL = f"https://kauth.kakao.com/oauth/authorize?client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code"

print("아래 URL을 브라우저에서 열고, 인증 코드(code)를 복사하세요:")
print(AUTH_URL)


아래 URL을 브라우저에서 열고, 인증 코드(code)를 복사하세요:
https://kauth.kakao.com/oauth/authorize?client_id=a3f4ffdc5ff3698dfa560ece7db9ab0e&redirect_uri=https://localhost&response_type=code


In [6]:
import requests
import json

REST_API_KEY = "a3f4ffdc5ff3698dfa560ece7db9ab0e"  # 카카오 개발자 센터에서 발급한 REST API 키
REDIRECT_URI = "https://localhost"
AUTH_CODE = "-HKq7o3I3MQjtn7xdUDBqrZgpb1djQFiTthLlLuGINpRBSgT8yJgdAAAAAQKKcjZAAABlYfwKhptZc76WqiBKA"  # 처음 한 번만 발급받은 코드

TOKEN_URL = "https://kauth.kakao.com/oauth/token"

# 📌 Step 2: Access Token & Refresh Token 요청
data = {
    "grant_type": "authorization_code",
    "client_id": REST_API_KEY,
    "redirect_uri": REDIRECT_URI,
    "code": AUTH_CODE
}

response = requests.post(TOKEN_URL, data=data)
tokens = response.json()

# 📌 Step 3: JSON 형식으로 토큰 저장
with open("kakao_token.json", "w") as file:
    json.dump(tokens, file, indent=4)  # JSON 형식 유지

print("✅ Access Token:", tokens.get("access_token"))
print("🔄 Refresh Token:", tokens.get("refresh_token"))

✅ Access Token: Daw3dXnxHQqzwRIQ3xL-OK1UcQ3P2NY_AAAAAQoXAVAAAAGVh_BkK5CBbdpZdq0Z
🔄 Refresh Token: sLGkP8lQgrfak60Eo6r0yKT2_SLTKtYWAAAAAgoXAVAAAAGVh_BkJJCBbdpZdq0Z


In [7]:
import requests
import json

REST_API_KEY = "a3f4ffdc5ff3698dfa560ece7db9ab0e"
TOKEN_URL = "https://kauth.kakao.com/oauth/token"

# 📌 Step 1: 저장된 토큰 불러오기
def load_tokens():
    try:
        with open("kakao_token.json", "r") as file:
            tokens = json.load(file)  # JSON 파일을 읽기
        return tokens
    except FileNotFoundError:
        print("❌ 토큰 파일이 없습니다. 먼저 Access Token을 발급하세요.")
        return None
    except json.JSONDecodeError:
        print("❌ JSON 파일이 손상되었습니다. 다시 Access Token을 발급하세요.")
        return None

# 📌 Step 2: 새로운 Access Token 발급
def refresh_access_token():
    tokens = load_tokens()
    if not tokens:
        return None

    refresh_token = tokens.get("refresh_token")
    
    data = {
        "grant_type": "refresh_token",
        "client_id": REST_API_KEY,
        "refresh_token": refresh_token
    }

    response = requests.post(TOKEN_URL, data=data)
    new_tokens = response.json()

    if "access_token" in new_tokens:
        tokens["access_token"] = new_tokens["access_token"]

        # 새로운 Access Token 저장
        with open("kakao_token.json", "w") as file:
            json.dump(tokens, file, indent=4)  # JSON 형식 유지

        print("✅ 새 Access Token 발급 완료!")
        return tokens["access_token"]
    else:
        print("❌ Refresh Token이 만료되었습니다. 다시 인증하세요.")
        return None

# 📌 Step 3: 테스트 실행
access_token = refresh_access_token()
print("사용할 Access Token:", access_token)


✅ 새 Access Token 발급 완료!
사용할 Access Token: RjilExvGNW4QGkmRbbdKjLMynaH1yptlAAAAAQoXFO4AAAGVh_Bt3pCBbdpZdq0Z


In [8]:
import requests
import json
from bs4 import BeautifulSoup

# 📌 카카오 API 정보 설정
REST_API_KEY = "a3f4ffdc5ff3698dfa560ece7db9ab0e"  # 카카오 개발자 센터에서 발급한 키
TOKEN_URL = "https://kauth.kakao.com/oauth/token"

# 📌 Access Token 자동 갱신
def refresh_access_token():
    try:
        with open("kakao_token.json", "r") as file:
            tokens = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        print("❌ 토큰 파일이 없거나 손상됨. Access Token을 먼저 발급하세요.")
        return None

    refresh_token = tokens.get("refresh_token")
    
    data = {
        "grant_type": "refresh_token",
        "client_id": REST_API_KEY,
        "refresh_token": refresh_token
    }

    response = requests.post(TOKEN_URL, data=data)
    new_tokens = response.json()

    if "access_token" in new_tokens:
        tokens["access_token"] = new_tokens["access_token"]

        # 새로운 Access Token 저장
        with open("kakao_token.json", "w") as file:
            json.dump(tokens, file, indent=4)

        print("✅ 새 Access Token 발급 완료!")
        return tokens["access_token"]
    else:
        print("❌ Refresh Token이 만료되었습니다. 다시 인증하세요.")
        return None

# 📌 뉴욕타임즈 'Read the Latest' 버튼에서 뉴스 URL 가져오기
def get_latest_news_url():
    url = "https://www.nytimes.com/newsletters/morning-briefing"
    response = requests.get(url)

    if response.status_code != 200:
        print("❌ 뉴욕타임즈 페이지 요청 실패!")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    button = soup.find("a", string="Read the Latest")

    if button:
        latest_news_url = button["href"]
        if latest_news_url.startswith("/"):
            latest_news_url = "https://www.nytimes.com" + latest_news_url  # 상대 경로 처리
        return latest_news_url
    else:
        print("❌ 'Read the Latest' 버튼을 찾을 수 없음")
        return None

# 📌 카카오톡으로 뉴스 URL 전송
def send_kakao_message(news_url):
    access_token = refresh_access_token()
    if not access_token:
        return
    
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    message = f"📢 뉴욕타임즈 Morning Briefing\n\n{news_url}"
    data = {
        "template_object": '{"object_type":"text","text":"' + message + '","link":{}}'
    }

    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        print("✅ 카카오톡 메시지 전송 성공!")
    else:
        print("❌ 전송 실패:", response.json())

# 📌 실행: 뉴스 URL 가져와서 카톡으로 전송
news_url = get_latest_news_url()
if news_url:
    send_kakao_message(news_url)


✅ 새 Access Token 발급 완료!
❌ 전송 실패: {'msg': '[auto_letter] App disabled [talk_message] scopes for [TALK_MEMO_DEFAULT_SEND] API on developers.kakao.com. Enable it first.', 'code': -3}
